# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.kinetics_bounded import kinetic_block_rule
from physics.energy_bounded import energy_block_rule
from physics.VLE_bounded_MPCC_T import VLE_block_rule
from physics.MPCC_T import dew_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals,initialize=3000) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

# Construct Individual Blocks

In [5]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [6]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [7]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.T_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}-\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [8]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [9]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

## MPCC: T

In [10]:
model.MPCC = pe.Block(rule=dew_block_rule)

> Importing dew Blocks......
> Adding the following local variable:
------------------------------------
| MPCC.x
| MPCC.y
| MPCC.T
| MPCC.P
| MPCC.f_V
| MPCC.f_L
| MPCC.beta
| MPCC.s_L
| MPCC.s_V
| MPCC.epi
------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.VLE_block.n_ave
| MPCC.VLE_block.n_ave_cal
| MPCC.VLE_block.Hen
| MPCC.VLE_block.Hen0
| MPCC.VLE_block.gamma
| MPCC.VLE_block.P_sat
| MPCC.VLE_block.P_sat_Y
| MPCC.VLE_block.P_sat_dY_inf
| MPCC.VLE_block.P_sat_dY0
| MPCC.VLE_block.Hen_ref
| MPCC.VLE_block.Hen0_ref
| MPCC.VLE_block.gamma_ref
| MPCC.VLE_block.V_L
| MPCC.VLE_block.V_L_dY_inf
| MPCC.VLE_block.V_L_dY0
| MPCC.VLE_block.poynting
--------------------------------------------------



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [11]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [12]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [13]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [14]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [15]:
model.cat.fix(3000)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(1)
model.z['CO'].fix(1/(1+0.8)-0/2)
model.z['H2'].fix(0.8/(1+0.8)-0/2)
model.z['C30H62'].fix(0)
model.VLE_block.n_ave.fix(20)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 1895
Active Inequality Constraints:	 1
Active Variables:		 2071
Fixed Variables:		 173
DOF:				 3


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(300+273.15)
# model.T.fix(524)

In [18]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.cat,sense=pe.minimize)
# model.obj = pe.Objective(expr = model.T - model.MPCC.pf ,sense=pe.maximize)
model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  74 -5.2189960e+02 2.14e-01 9.67e+01  -1.0 9.82e+03    -  1.12e-02 2.90e-04h  6
  75 -5.2191691e+02 2.14e-01 1.11e+02  -1.0 9.81e+03    -  4.65e-03 2.81e-04h  6
  76 -5.2195047e+02 2.15e-01 1.95e+02  -1.0 9.80e+03    -  3.89e-02 5.47e-04h  5
  77 -5.2198213e+02 2.15e-01 4.55e+02  -1.0 9.77e+03    -  4.60e-03 5.17e-04h  5
  78 -5.2201195e+02 2.15e-01 2.29e+03  -1.0 9.74e+03    -  9.21e-02 4.89e-04h  5
  79 -5.2250379e+02 1.13e+00 1.08e+06  -1.0 3.14e+04    -  4.92e-03 8.28e-03w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -5.2253586e+02 1.40e+00 1.13e+06  -1.0 1.96e+05    -  4.27e-03 6.48e-04w  1
  81 -5.2254360e+02 1.43e+00 1.14e+06  -1.0 2.45e+05    -  1.66e-04 1.64e-04w  1
  82 -5.2201579e+02 2.15e-01 3.62e+03  -1.0 2.47e+05    -  4.92e-03 6.47e-05h  7
  83 -5.2201779e+02 2.15e-01 1.30e+04  -1.0 4.49e+04    -  2.28e-02 3.40e-05h  9
  84r-5.2201779e+02 2.15e-01 1.00e+03  -0.7 0.00e+00    -  0.00e+00 3.32e-07R 16
  85r-5.2201809e+02 1.59e-01

 183r-5.1905593e+02 2.46e-04 6.73e+02  -1.0 0.00e+00    -  0.00e+00 2.93e-07R 17
 184r-5.1905609e+02 2.46e-04 8.38e+03  -1.0 1.20e+03    -  3.98e-03 1.18e-04f  1
 185r-5.1906530e+02 5.40e-02 6.93e+03  -1.0 3.25e+01    -  1.18e-01 9.93e-02f  1
 186r-5.1907429e+02 1.00e-01 8.27e+03  -1.0 1.54e+01    -  5.42e-01 1.39e-01f  1
 187r-5.1915777e+02 1.10e+00 3.30e+03  -1.0 2.86e+01    -  6.39e-01 6.97e-01f  1
 188r-5.1938231e+02 1.11e+00 6.71e+01  -1.0 4.61e+01    -  9.88e-01 1.00e+00f  1
 189r-5.1973056e+02 5.08e-02 1.41e+01  -1.0 5.49e+01    -  9.91e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r-5.1973616e+02 3.29e-04 4.35e-03  -1.0 9.36e-01    -  1.00e+00 1.00e+00h  1
 191r-5.1935842e+02 1.60e+00 2.52e+02  -3.8 1.04e+02    -  5.44e-01 5.71e-01f  1
 192r-5.1889965e+02 1.73e+00 1.13e+02  -3.8 1.79e+02    -  4.10e-01 4.04e-01f  1
 193r-5.1848109e+02 1.20e+00 6.26e+01  -3.8 1.58e+02    -  4.16e-01 4.15e-01f  1
 194r-5.1822553e+02 6.91e-01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280 -5.2384139e+02 5.63e-02 1.42e+06  -1.7 5.89e+01    -  8.39e-01 1.00e+00f  1
 281 -5.2396753e+02 6.15e-02 2.55e+05  -1.7 6.52e+01    -  1.00e+00 1.00e+00H  1
 282 -5.2392395e+02 1.19e-01 3.42e+04  -1.7 2.47e+01    -  1.00e+00 1.00e+00f  1
 283 -5.2394148e+02 1.61e-03 4.94e+03  -1.7 9.94e+00    -  1.00e+00 1.00e+00h  1
 284 -5.2394243e+02 2.21e-05 1.46e+01  -1.7 5.35e-01    -  1.00e+00 1.00e+00h  1
 285 -5.2397492e+02 8.48e-02 4.08e+05  -3.8 3.08e+01    -  8.20e-01 5.99e-01f  1
 286 -5.2397714e+02 2.58e-05 4.88e+03  -3.8 1.25e+00    -  1.00e+00 1.00e+00h  1
 287 -5.2397717e+02 4.24e-09 4.07e-02  -3.8 1.61e-02    -  1.00e+00 1.00e+00h  1
 288 -5.2397732e+02 1.19e-07 4.73e+01  -8.6 8.59e-02    -  9.99e-01 9.96e-01f  1
 289 -5.2397732e+02 2.88e-10 7.61e-06  -8.6 2.58e-05    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 289

                                   (scaled)                 (unscaled)
Object

In [20]:
model.T.value

523.9773179920601

In [21]:
model.MPCC.T.value

523.8669145395334

In [22]:
model.VLE_block.T_VLE.value

523.8646947490765

In [23]:
model.L['out'].value

0.0

In [24]:
sum(model.MPCC.y[i].value for i in m.COMP_TOTAL)

1.0000000000000007

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}'.format(model.x[i].value,model.y[i].value))

# Iterative Solve for Data Analysis

In [ ]:
results_anchor = results
update_dual(pe,model)
model.solutions.store_to(results_anchor)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
catrange = [3000,1000,500,300,100,50,25,10]
Trange = np.linspace(300+273.15,200+273.15,26)

In [ ]:
rf_data_master = {}

for c in catrange:
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.cat ,sense=pe.minimize)

    model.solutions.load_from(results_anchor)
    update_dual(pe,model) 
       
    model.T.fix(300+273.15)
    model.cat.fixed = False
    model.cat.setlb(c)
    
    results_anchor = opt.solve(model,tee=False)
    model.solutions.store_to(results_anchor)
    update_dual(pe,model)
    
    model.del_component(model.obj)
    model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
    model.T.fixed = False
    model.cat.fixed = True
    
    
    print('\nWorking on Cat = {} kg'.format(model.cat.value))
    print('-'*108)
    
    rf_data = {}
    rf_data['x'] = {};rf_data['y'] = {};
    rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['P_VLE'] = []; 
    rf_data['s_L'] = [];
    for i in m.COMP_TOTAL:
        rf_data['x'][i] = []
        rf_data['y'][i] = []       

    for t in Trange:
        model.T.setub(t)
        results = opt.solve(model,tee=False)
        update_dual(pe,model)
        model.T.setub(300+273.15)
        print('Solved\t|T = {:.2f} K\t|V = {:.4f} kmol/s\t|L = {:.4f} kmol/s\t|s_L = {:.4f}\t|P_VLE = {:.1f}'.\
              format(model.T.value,model.V['out'].value,model.L['out'].value,model.MPCC.s_L.value,model.VLE_block.P_VLE.value))
    
        rf_data['T'].append(model.T.value)
        rf_data['Q'].append(model.Q_main.value)
        rf_data['V'].append(model.V['out'].value)
        rf_data['L'].append(model.L['out'].value)
        rf_data['P_VLE'].append(model.VLE_block.P_VLE.value)
        rf_data['s_L'].append(model.MPCC.s_L.value)

        for i in m.COMP_TOTAL:
            rf_data['x'][i].append(model.x[i].value)
            rf_data['y'][i].append(model.y[i].value)
        
    rf_data_master[c] = rf_data

# Optimization results across disappearing phases

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,rf_data_master[c]['V'])
ax1.set_ylabel('Vapor Flow Rate (kmol/s)',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.grid()

for c in catrange:
    ax2.plot(Trange,rf_data_master[c]['L'])
ax2.set_ylabel('Liquid Flow Rate (kmol/s)',fontsize=14)
ax2.set_xlabel('Temperature (K)',fontsize=14)
ax2.tick_params(labelsize=14)
ax2.grid()

ax2.legend(catrange,fontsize=14)
fig.suptitle('Vapor and Liquid Flow Rate',fontsize=18)

plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,np.divide(rf_data_master[c]['V'],np.add(rf_data_master[c]['V'],rf_data_master[c]['L'])))
ax1.set_ylabel('Vapor Fraction',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.legend(catrange,fontsize=14)
ax1.set_title('Vapor Fraction',fontsize=18)
ax1.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,rf_data_master[c]['P_VLE'])
ax1.set_ylabel(r'$P_{VLE}$ value',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.legend(catrange,fontsize=14)
ax1.set_title(r'$P_{VLE}$ (Bar)',fontsize=18)
ax1.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
for c in catrange:
    ax1.plot(Trange,np.multiply(rf_data_master[c]['s_L'],rf_data_master[c]['L']))
ax1.set_ylabel(r'$s_L \cdot L$ value',fontsize=14)
ax1.set_xlabel('Temperature (K)',fontsize=14)
ax1.tick_params(labelsize=14)
ax1.legend(catrange,fontsize=14)
# ax1.set_ylim(2.49e-5,2.51e-5)
ax1.set_title(r'$s_L \cdot L$',fontsize=18)
ax1.grid()
plt.show()